In [30]:
import numpy as np

In [31]:
X_train, y_train, X_test, y_test, X_dev, y_dev = np.load('./all_data.npy', allow_pickle = True)

In [32]:
def make_X(x):
    res = []
    x = x[767:]
    for i in range(0, x.shape[0], 768):
#         print(i)
        res.append(np.array(x[i:i+768]))
    print(np.array(res).shape)
    return np.array(res)

In [33]:
X_train, X_test, X_dev = make_X(X_train), make_X(X_test), make_X(X_dev)

In [34]:
num_classes = 7
timestep = 5

In [35]:
def expand_on_time(X, timestep = 20):
    tmp = np.reshape(X[0], (1, len(X[0])))
#     print(tmp.shape)
    tmp = np.repeat(tmp, timestep, axis = 0)
#     print(tmp.shape)
    ab = np.reshape(tmp, (1, len(tmp[:,:]), len(tmp[0])))
#     print(ab.shape)
#     print(X.shape)
    X = np.repeat(ab, X.shape[0], axis = 0)
#     print(X.shape)
    return X

In [36]:
X_train, X_test, X_dev = expand_on_time(X_train), expand_on_time(X_test), expand_on_time(X_dev)

In [37]:
X_train.shape, X_test.shape, X_dev.shape

((9989, 20, 768), (2610, 20, 768), (1109, 20, 768))

In [38]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM
from keras.optimizers import RMSprop, SGD

In [56]:
timestep = 20

model = Sequential()
model.add(LSTM(256, input_shape=(timestep, 768), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(128, return_sequences = True))
model.add(Dropout(0.4))
model.add(LSTM(64))
model.add(Dropout(0.5))
# model.add(Activation('softmax'))
model.add(Dense(128, activation = 'tanh'))
model.add(Dropout(0.4))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(num_classes, activation = 'sigmoid'))

sgd = SGD(lr=1e-2, decay=1e-6, momentum=0.9, nesterov=True)
#rms = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [57]:
model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_34 (LSTM)               (None, 20, 256)           1049600   
_________________________________________________________________
dropout_24 (Dropout)         (None, 20, 256)           0         
_________________________________________________________________
lstm_35 (LSTM)               (None, 20, 128)           197120    
_________________________________________________________________
dropout_25 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_36 (LSTM)               (None, 64)                49408     
_________________________________________________________________
dropout_26 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_16 (Dense)             (None, 128)             

In [58]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)


In [59]:
hist = model.fit(X_train, y_train, 
                 validation_data = (X_dev, y_dev), 
                 verbose=1, 
                 epochs=10, batch_size=32, 
                 shuffle=True, callbacks=[early_stopping])

Epoch 1/10
313/313 [==============================] - 60s 177ms/step - loss: 1.6264 - accuracy: 0.4532 - val_loss: 1.6437 - val_accuracy: 0.4238
Epoch 2/10
313/313 [==============================] - 22s 69ms/step - loss: 1.5602 - accuracy: 0.4708 - val_loss: 1.6328 - val_accuracy: 0.4238
Epoch 3/10
313/313 [==============================] - 20s 65ms/step - loss: 1.5586 - accuracy: 0.4664 - val_loss: 1.6307 - val_accuracy: 0.4238
Epoch 4/10
313/313 [==============================] - 20s 63ms/step - loss: 1.5472 - accuracy: 0.4731 - val_loss: 1.6381 - val_accuracy: 0.4238
Epoch 5/10
313/313 [==============================] - 20s 63ms/step - loss: 1.5639 - accuracy: 0.4631 - val_loss: 1.6273 - val_accuracy: 0.4238
Epoch 6/10
313/313 [==============================] - 20s 65ms/step - loss: 1.5456 - accuracy: 0.4704 - val_loss: 1.6297 - val_accuracy: 0.4238
Epoch 7/10
313/313 [==============================] - 20s 63ms/step - loss: 1.5480 - accuracy: 0.4703 - val_loss: 1.6356 - val_accuracy

In [60]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 1.5291389226913452
Test accuracy: 0.4812260568141937


In [62]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
class_names = ['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']


y_pred = model.predict(X_test)

y_pred = (y_pred > 0.5)
accuracy_score(y_test, y_pred, normalize=False)
print(classification_report(y_test, y_pred, target_names=class_names, digits=4))

              precision    recall  f1-score   support

       anger     0.1322    1.0000    0.2335       345
     disgust     0.0000    0.0000    0.0000        68
        fear     0.0000    0.0000    0.0000        50
         joy     0.1540    1.0000    0.2669       402
     neutral     0.4812    1.0000    0.6498      1256
     sadness     0.0000    0.0000    0.0000       208
    surprise     0.1077    1.0000    0.1944       281

   micro avg     0.2188    0.8751    0.3500      2610
   macro avg     0.1250    0.5714    0.1921      2610
weighted avg     0.2844    0.8751    0.4056      2610
 samples avg     0.2188    0.8751    0.3500      2610



/home/anuragpendyala.31/anaconda3/envs/ser2/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [63]:
from sklearn.metrics import confusion_matrix
#Get the confusion matrix


#array([[1, 0, 0],
#   [1, 0, 0],
#   [0, 1, 2]])
cm =confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
#Now the normalize the diagonal entries
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
#array([[1.        , 0.        , 0.        ],
#      [1.        , 0.        , 0.        ],
#      [0.        , 0.33333333, 0.66666667]])

#The diagonal entries are the accuracies of each class
cm.diagonal()

array([1., 0., 0., 0., 0., 0., 0.])